# 0 Imports

## 0.1. Libraries

In [47]:
# data manipulation
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import (to_timestamp, count, when, max, col, round, kurtosis, skewness,
                                   date_format, weekofyear, dayofweek)
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType

# data visualization
from matplotlib import pyplot as plt
import seaborn as sns

# other
from typing import Dict
import os
from IPython.display import Image
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

## 0.2. Helper functions

In [2]:
def calculate_bounds(df: pyspark.sql.dataframe.DataFrame) -> Dict[str, Dict[str, float]]:
    '''Calculate bounds for numerical columns in a dataframe.
    
    Parameters
    ----------
    df : pyspark DataFrame
       DataFrame to perform the operations.
    
    Returns
    ----------
    A dictionary containing the first and third quantiles and the lower and upper bounds of each column.
    '''
    # calculate quantiles
    bounds = {col: dict(zip(['q25', 'q75'], df.approxQuantile(col, [0.25, 0.75], 0)))
              for col, dt in zip(df.columns, df.dtypes) if dt[1] == 'float'}
    
    for col in bounds:
        # calculate the interquantile
        iqr = bounds[col]['q75'] - bounds[col]['q25']
        
        # calculate the cut off
        cut_off = iqr * 1.5
        
        # calculate the minimum and maximum bound
        bounds[col]['lower'] = bounds[col]['q25'] - cut_off
        bounds[col]['upper'] = bounds[col]['q75'] + cut_off

    return bounds

## 0.3. Data acquisition
Data source: [Kaggle](https://www.kaggle.com/competitions/airbnb-recruiting-new-user-bookings)

In [3]:
MAX_MEMORY = '6g'

# instantiate spark session object
spark = SparkSession.builder \
                    .master('local[*]') \
                    .config("spark.submit.deployMode", 'client') \
                    .config('spark.executor.instances', '16') \
                    .config('spark.driver.memory', MAX_MEMORY) \
                    .config('spark.executor.memory', MAX_MEMORY) \
                    .config('spark.executor.memoryOverhead', MAX_MEMORY) \
                    .appName('airbnb-first-booking-prediction') \
                    .getOrCreate()

22/05/30 08:45:23 WARN Utils: Your hostname, archlinux resolves to a loopback address: 127.0.1.1; using 192.168.0.21 instead (on interface enp3s0)
22/05/30 08:45:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/30 08:45:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!ls -lh ../data/raw

total 633M
-rw-r--r-- 1 qwoek qwoek  12K Oct 16  2015 age_gender_bkts.csv
-rw-r--r-- 1 qwoek qwoek  632 Oct 16  2015 countries.csv
-rw-r--r-- 1 qwoek qwoek 603M Dec  6  2015 sessions.csv
-rw-r--r-- 1 qwoek qwoek 6.5M May 16  2018 test_users.csv
-rw-r--r-- 1 qwoek qwoek  24M Dec 10  2015 train_users_2.csv


In [5]:
!head -n 10 ../data/raw/train_users_2.csv

id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
gxn3p5htnn,2010-06-28,20090319043255,,-unknown-,,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
820tgsjxq7,2011-05-25,20090523174809,,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
osr2jwljor,2010-01-01,20100101215619,2010-01-02,-unknown-,,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US
lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,

In [6]:
# define dataset schema
schema_df_train = '''
    id STRING,
    date_account_created TIMESTAMP,
    timestamp_first_active STRING,
    date_first_booking STRING,
    gender STRING,
    age FLOAT,
    signup_method STRING,
    signup_flow INTEGER,
    language STRING,
    affiliate_channel STRING,
    affiliate_provider STRING,
    first_affiliate_tracked STRING,
    signup_app STRING,
    first_device_type STRING,
    first_browser STRING,
    country_destination STRING
'''

# read dataset
df_train = spark.read.csv(path='../data/raw/train_users_2.csv', schema=schema_df_train, header=True)

# format date columns
df_train = df_train.withColumn('timestamp_first_active', to_timestamp('timestamp_first_active', 'yyyyMMddHHmmss'))
df_train = df_train.withColumn('date_first_booking', to_timestamp('date_first_booking', 'yyyy-MM-dd'))

# check it out
df_train.show(5)

+----------+--------------------+----------------------+-------------------+---------+----+-------------+-----------+--------+-----------------+------------------+-----------------------+----------+-----------------+-------------+-------------------+
|        id|date_account_created|timestamp_first_active| date_first_booking|   gender| age|signup_method|signup_flow|language|affiliate_channel|affiliate_provider|first_affiliate_tracked|signup_app|first_device_type|first_browser|country_destination|
+----------+--------------------+----------------------+-------------------+---------+----+-------------+-----------+--------+-----------------+------------------+-----------------------+----------+-----------------+-------------+-------------------+
|gxn3p5htnn| 2010-06-28 00:00:00|   2009-03-19 04:32:55|               null|-unknown-|null|     facebook|          0|      en|           direct|            direct|              untracked|       Web|      Mac Desktop|       Chrome|                N

In [7]:
!head -n 10 ../data/raw/sessions.csv

user_id,action,action_type,action_detail,device_type,secs_elapsed
d1mm9tcy42,lookup,,,Windows Desktop,319.0
d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
d1mm9tcy42,lookup,,,Windows Desktop,301.0
d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
d1mm9tcy42,lookup,,,Windows Desktop,435.0
d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,7703.0
d1mm9tcy42,lookup,,,Windows Desktop,115.0
d1mm9tcy42,personalize,data,wishlist_content_update,Windows Desktop,831.0
d1mm9tcy42,index,view,view_search_results,Windows Desktop,20842.0


In [8]:
# define dataset schema
schema_df_sessions = StructType([
  StructField('user_id', StringType(), True),
  StructField('action', StringType(), True),
  StructField('action_type', StringType(), True),
  StructField('action_detail', StringType(), True),
  StructField('device_type', StringType(), True),
  StructField('secs_elapsed', FloatType(), True)
])

# read dataset
df_sessions = spark.read.csv(path='../data/raw/sessions.csv', schema=schema_df_sessions, header=True)

# check it out
df_sessions.show(5)

+----------+--------------+-----------+-------------------+---------------+------------+
|   user_id|        action|action_type|      action_detail|    device_type|secs_elapsed|
+----------+--------------+-----------+-------------------+---------------+------------+
|d1mm9tcy42|        lookup|       null|               null|Windows Desktop|       319.0|
|d1mm9tcy42|search_results|      click|view_search_results|Windows Desktop|     67753.0|
|d1mm9tcy42|        lookup|       null|               null|Windows Desktop|       301.0|
|d1mm9tcy42|search_results|      click|view_search_results|Windows Desktop|     22141.0|
|d1mm9tcy42|        lookup|       null|               null|Windows Desktop|       435.0|
+----------+--------------+-----------+-------------------+---------------+------------+
only showing top 5 rows



In [9]:
# join datasets
df = df_train.join(df_sessions, on=[df_train.id == df_sessions.user_id], how='inner')

# drop repetead column
df = df.drop('user_id')

# check it out
df.show(3)

+----------+--------------------+----------------------+-------------------+---------+----+-------------+-----------+--------+-----------------+------------------+-----------------------+----------+-----------------+-------------+-------------------+--------------------+-----------+--------------------+---------------+------------+
|        id|date_account_created|timestamp_first_active| date_first_booking|   gender| age|signup_method|signup_flow|language|affiliate_channel|affiliate_provider|first_affiliate_tracked|signup_app|first_device_type|first_browser|country_destination|              action|action_type|       action_detail|    device_type|secs_elapsed|
+----------+--------------------+----------------------+-------------------+---------+----+-------------+-----------+--------+-----------------+------------------+-----------------------+----------+-----------------+-------------+-------------------+--------------------+-----------+--------------------+---------------+------------

In [10]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date_account_created: timestamp (nullable = true)
 |-- timestamp_first_active: timestamp (nullable = true)
 |-- date_first_booking: timestamp (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: float (nullable = true)
 |-- signup_method: string (nullable = true)
 |-- signup_flow: integer (nullable = true)
 |-- language: string (nullable = true)
 |-- affiliate_channel: string (nullable = true)
 |-- affiliate_provider: string (nullable = true)
 |-- first_affiliate_tracked: string (nullable = true)
 |-- signup_app: string (nullable = true)
 |-- first_device_type: string (nullable = true)
 |-- first_browser: string (nullable = true)
 |-- country_destination: string (nullable = true)
 |-- action: string (nullable = true)
 |-- action_type: string (nullable = true)
 |-- action_detail: string (nullable = true)
 |-- device_type: string (nullable = true)
 |-- secs_elapsed: float (nullable = true)



If there is a folder called interim, delete it and split the data, otherwise, just split the data.

In [11]:
PATH_INTERIM = '../data/interim/'

if os.path.isdir(PATH_INTERIM):
    try:
        # delete interim folder
        os.system(f'rm -rf {PATH_INTERIM}')
    except OSError as Error:
        print(Error)
    else:
        # split dataset into 16 pieces to leverage spark's engine
        df.repartition(16).write.parquet(PATH_INTERIM)
        
        # check it out (one of the files is a success message)
        !ls -lx  ../data/interim | wc -l
else:
    df.repartition(16).write.parquet(PATH_INTERIM)
    !ls -lx  ../data/interim | wc -l

17


In [12]:
# read splited data
df = spark.read.parquet(PATH_INTERIM)

# 1. Data cleansing and description

## Index

Column | Meaning
---|--------
id | user id
date_account_created | the date of account creation
timestamp_first_active | timestamp of the first activity, note that it can be earlier than date_account_created or date_first_booking because a user can search before signing up
date_first_booking | date of first booking
gender |
age |
signup_method |
signup_flow | the page a user came to signup up from
language | international language preference
affiliate_channel | what kind of paid marketing
affiliate_provider | where the marketing is e.g. google, craigslist, other
first_affiliate_tracked | whats the first marketing the user interacted with before the signing up
signup_app |
first_device_type |
first_browser |
country_destination | this is the target variable you are to predict
action |
action_type |
action_detail |
device_type |
secs_elapsed |

## 1.1. Data dimensions

In [17]:
# check dataset shape
print(f'Number of rows: {df.count()}\n', \
      + f'Number of columns: {len(df.columns)}')

Number of rows: 5537957
Number of columns: 21


## 1.2. Duplicate data

In [18]:
# check if there is duplicate data
if df.count() > df.dropDuplicates().count():
    print(f'There were {df.count() - df.dropDuplicates().count()} duplicate records.')
    df = df.dropDuplicates()
else:
    print('There is not duplicate data.')

There were 149501 duplicate records.


## 1.3. Missing data

In [19]:
# check if there is null values
df.select([count(when(df[x].isNull(), x)).alias(x) for x in df.columns]).show()

+---+--------------------+----------------------+------------------+------+-------+-------------+-----------+--------+-----------------+------------------+-----------------------+----------+-----------------+-------------+-------------------+------+-----------+-------------+-----------+------------+
| id|date_account_created|timestamp_first_active|date_first_booking|gender|    age|signup_method|signup_flow|language|affiliate_channel|affiliate_provider|first_affiliate_tracked|signup_app|first_device_type|first_browser|country_destination|action|action_type|action_detail|device_type|secs_elapsed|
+---+--------------------+----------------------+------------------+------+-------+-------------+-----------+--------+-----------------+------------------+-----------------------+----------+-----------------+-------------+-------------------+------+-----------+-------------+-----------+------------+
|  0|                   0|                     0|           2976349|     0|2055829|            0|

**Action**: Drop rows.

**Reason**: Too many non-available values for a few rows.

**Columns**:
* secs_elapsed (73,815) 
* action (51,294)
* first_affiliate_tracked (1,331)

In [20]:
df = df.na.drop(how='any', subset=['secs_elapsed', 'action', 'first_affiliate_tracked'])

**Action**: Drop columns.

**Reason**: There's no data here, basically.

**Columns**:
* age (5,388,456)

In [22]:
df = df.drop('age')

**Action**: Fill in.

**Reason**: Already expected to not be available.

**Columns**:
* action_type (587,088)
* action_detail (587,088)

In [23]:
df = df.na.fill(value='-unknown-', subset=['action_type', 'action_detail'])

**Reason**: I noticed that people who don't have a first booking date set yet are those who haven't even booked a place yet.

**Columns**:
* date_first_booking (2,976,349)

In [24]:
df.select(['date_first_booking']).where(df['date_first_booking'].isNull()).count() == \
df.select(['country_destination']).where(df['country_destination'] == 'NDF').count()

True

In [25]:
# get maximum first booking date
max_date_first_booking = df.select('date_first_booking') \
                            .agg({'date_first_booking': 'max'}) \
                            .collect()[0][0].timestamp()

# fill out
df = df.na.fill(value=max_date_first_booking, subset=['date_first_booking'])

In [26]:
# check number of rows and column after the treatment
print(f'Number of rows: {df.count()}\nNumber of columns: {len(df.columns)}')

Number of rows: 5273754
Number of columns: 20


## 1.4. Outliers

### Inter-Quartile Range (IQR) method

In [27]:
# calculate numeric columns bounds
bounds = calculate_bounds(df)

# check it out
bounds

{'secs_elapsed': {'q25': 279.0,
  'q75': 8755.0,
  'lower': -12435.0,
  'upper': 21469.0}}

In [28]:
# remove outliers
for c, dt in zip(df.columns, df.dtypes):
    if dt[1] == 'float':
        df = df.select('*') \
               .filter(col(c).between(bounds[c]['lower'], bounds[c]['upper']))

# check new number of rows
df.count()

4484929

## 1.5. Imbalance

As it is a multi-class classification we have many possibly destinations to deal with, 
and in this case we also have a huge data imbalance.

In [29]:
# check data imbalance
df.groupBy('country_destination') \
  .count() \
  .orderBy('count', ascending=False) \
  .withColumn('count_percentage', round(100*(col('count') / df.count()), 2)).show()

+-------------------+-------+----------------+
|country_destination|  count|count_percentage|
+-------------------+-------+----------------+
|                NDF|2486115|           55.43|
|                 US|1349440|           30.09|
|              other| 258863|            5.77|
|                 FR| 117940|            2.63|
|                 IT|  82213|            1.83|
|                 ES|  54320|            1.21|
|                 GB|  51931|            1.16|
|                 CA|  32049|            0.71|
|                 DE|  18985|            0.42|
|                 NL|  18206|            0.41|
|                 AU|   9794|            0.22|
|                 PT|   5073|            0.11|
+-------------------+-------+----------------+



## 1.6. Descriptive statistics

### 1.6.1. Numerical attributes

In [30]:
# describe numerical columns
df.select(['signup_flow', 'secs_elapsed']).summary().show()

+-------+-----------------+------------------+
|summary|      signup_flow|      secs_elapsed|
+-------+-----------------+------------------+
|  count|          4484929|           4484929|
|   mean| 5.50373350391946|2926.8514027758297|
| stddev|9.824296513960798| 4655.576726729597|
|    min|                0|               0.0|
|    25%|                0|             182.0|
|    50%|                0|             845.0|
|    75%|               12|            3080.0|
|    max|               25|           21469.0|
+-------+-----------------+------------------+



In [31]:
((21469 / 60)/ 60)

5.963611111111111

In [32]:
# plus calculate kurtosis and skewness
df.select(kurtosis('signup_flow').alias('signup_flow_kurtosis'), \
          skewness('signup_flow').alias('signup_flow_skewness'), \
          kurtosis('secs_elapsed').alias('secs_elapsed_kurtosis'), \
          skewness('secs_elapsed').alias('secs_elapsed_skewness')).show()

+--------------------+--------------------+---------------------+---------------------+
|signup_flow_kurtosis|signup_flow_skewness|secs_elapsed_kurtosis|secs_elapsed_skewness|
+--------------------+--------------------+---------------------+---------------------+
|-0.14639712784852232|  1.3165766326768453|   3.8406743105607104|    2.134835842785242|
+--------------------+--------------------+---------------------+---------------------+



### 1.6.2. Categorical attributes

#### Unique values

In [33]:
# compute all unique values of each string column
unique_values = {col: df.select(col).distinct().collect()
                 for col, dt in zip(df.columns, df.dtypes) if dt[1] == 'string' and dt[0] != 'id'}

# check it out
for key, _ in unique_values.items():
    # print the column name and how many layers it has
    print(f'Column name: {key} | Count: {len(unique_values[key])}')
    
    # print layers
    for value in unique_values[key]:
        print(value.asDict()[key])
    print('\n')

Column name: gender | Count: 4
MALE
OTHER
-unknown-
FEMALE


Column name: signup_method | Count: 3
basic
facebook
google


Column name: language | Count: 24
en
pl
pt
ko
cs
tr
de
is
es
el
it
sv
nl
hu
ca
ru
th
no
zh
fr
ja
id
da
fi


Column name: affiliate_channel | Count: 8
api
direct
content
remarketing
other
seo
sem-non-brand
sem-brand


Column name: affiliate_provider | Count: 17
craigslist
bing
gsp
naver
direct
baidu
other
yandex
padmapper
vast
email-marketing
meetup
daum
facebook
facebook-open-graph
yahoo
google


Column name: first_affiliate_tracked | Count: 7
tracked-other
untracked
marketing
local ops
linked
omg
product


Column name: signup_app | Count: 4
iOS
Moweb
Android
Web


Column name: first_device_type | Count: 9
Android Tablet
iPad
iPhone
Windows Desktop
SmartPhone (Other)
Android Phone
Mac Desktop
Other/Unknown
Desktop (Other)


Column name: first_browser | Count: 35
Silk
SiteKiosk
Mobile Firefox
Avant Browser
CoolNovo
RockMelt
Sogou Explorer
TheWorld Browser
Yandex.Bro

# 2. Feature engineering

## 2.1. Phenomenon modeling

In [34]:
# date_account_created
df = df.withColumn('year_account_created', date_format(df.date_account_created, 'yyyy')) \
       .withColumn('month_account_created', date_format(df.date_account_created, 'MM')) \
       .withColumn('week_of_year_account_created', weekofyear(df.date_account_created)) \
       .withColumn('day_of_week_account_created', dayofweek(df.date_account_created)) \
       .withColumn('day_account_created', date_format(df.date_account_created, 'dd'))

In [35]:
# timestamp_first_active
df = df.withColumn('timestamp_first_active_year', date_format(df.timestamp_first_active, 'yyyy')) \
       .withColumn('timestamp_first_active_month', date_format(df.timestamp_first_active, 'MM')) \
       .withColumn('timestamp_first_active_week_of_year', weekofyear(df.timestamp_first_active)) \
       .withColumn('timestamp_first_active_day_of_week', dayofweek(df.timestamp_first_active)) \
       .withColumn('timestamp_first_active_day', date_format(df.timestamp_first_active, 'dd'))

In [36]:
# date_first_booking
df = df.withColumn('year_first_booking', date_format(df.date_first_booking, 'yyyy')) \
       .withColumn('month_first_booking', date_format(df.date_first_booking, 'MM')) \
       .withColumn('week_of_year_first_booking', weekofyear(df.date_first_booking)) \
       .withColumn('day_of_week_first_booking', dayofweek(df.date_first_booking)) \
       .withColumn('day_first_booking', date_format(df.date_first_booking, 'dd'))

In [37]:
# drop source columns
df = df.drop('date_account_created', 'timestamp_first_active', 'date_first_booking')

In [38]:
# secs_elapsed_times_singup_flow
df = df.withColumn('secs_elapsed_times_singup_flow', df.secs_elapsed * df.signup_flow)

# 3. Exploratory data analysis

## 3.1. Bivariate analysis

### Problem mind map

In [ ]:
Image('../reports/figures/mind-map')

### Hypotheses list

### 3.1.1. Hypothesis 01

## 3.2. Multivariate analysis

### 3.2.1. Numerical variables

### 3.2.2. Categorical variables